In [1]:
%load_ext autoreload
%autoreload 2
import source as sc
from sig import Signal
from strategy import Strategy

## 获取历史K线

In [2]:
df = sc.get_ts_stock_data('SZ000001', '20210101')

## 定义交易信号

#### 买入

In [3]:
class LongSig(Signal):
    def evaluate(self):
        """昨天涨就买"""
        if self._kline['change'].iloc[0] > 0:
            return True
        else:
            return False

#### 止盈

In [4]:
class TpSig(Signal):
    def evaluate(self):
        """昨天跌就卖"""
        if self._kline['change'].iloc[0] < 0:
            return True
        else:
            return False

#### 止损

In [5]:
class LsSig(Signal):
    def evaluate(self):
        """昨天跌就卖"""
        if self._kline['change'].iloc[0] < 0:
            return True
        else:
            return False

## 指定策略

In [11]:
s = Strategy()
trade_ls = []

for i in range(len(df)-2, -1, -1):
    date = df['trade_date'].iloc[i] # 第n天
    kline = df[i+1:] # 前n-1天的K线
    
    # 信号
    long_sig = LongSig(kline)
    tp_sig = TpSig(kline)
    ls_sig = LsSig(kline)
    
    op_dic = s.open_position(long_sig, df['open'].iloc[i], 100)
    tp_dic = s.take_profit(tp_sig, df['open'].iloc[i])
    #ls_dic = s.open_position(long_sig, df['open'].iloc[i], 100)
    
    operation = {'date': date} # 实际操作
    operation.update(op_dic.copy())
    operation.update(tp_dic.copy())
    
    trade_ls.append(operation)

## 回测结果

In [12]:
trade_ls

[{'date': '20210105'},
 {'date': '20210106'},
 {'date': '20210107',
  'operation': 'buy',
  'price': 19.52,
  'lot': 100,
  'position': 100},
 {'date': '20210108',
  'operation': 'buy',
  'price': 19.9,
  'lot': 100,
  'position': 200},
 {'date': '20210111',
  'operation': 'sell_tp',
  'price': 20.0,
  'lot': -200,
  'position': 0},
 {'date': '20210112',
  'operation': 'buy',
  'price': 20.39,
  'lot': 100,
  'position': 100},
 {'date': '20210113',
  'operation': 'buy',
  'price': 21.0,
  'lot': 100,
  'position': 200},
 {'date': '20210114',
  'operation': 'sell_tp',
  'price': 20.68,
  'lot': -200,
  'position': 0},
 {'date': '20210115'},
 {'date': '20210118',
  'operation': 'buy',
  'price': 21.2,
  'lot': 100,
  'position': 100},
 {'date': '20210119',
  'operation': 'buy',
  'price': 22.51,
  'lot': 100,
  'position': 200},
 {'date': '20210120',
  'operation': 'sell_tp',
  'price': 22.15,
  'lot': -200,
  'position': 0},
 {'date': '20210121',
  'operation': 'buy',
  'price': 22.5,
 